# Notebook 5 — Performance & Risk Diagnostics

**Goal.** Evaluate the Opening Range strategy using the quant metrics you listed (annualized return, Sharpe, Sortino, alpha/beta, drawdowns, Calmar, consistency) plus any related ratios we may add along the way.

**Plan for this notebook**
- **5.1** Setup & data prep (load backtest, build daily return series).
- **5.2** Compounding & annualized return (WIP).
- **5.3** Sharpe + Sortino (WIP).
- **5.4** Alpha/Beta vs benchmark (WIP).
- **5.5** Drawdowns, Calmar, consistency checks (WIP).

These sections will be filled one at a time so we can review each metric block together.


### 5.1 — Setup & data prep (read this first)

*Purpose:* pull the cleaned trade history (`reports/tables/backtest_daily_net.csv`) created in Notebook 3, compute the daily net returns, and stash a tidy DataFrame (`perf`) we will use everywhere else.

*What this section does:*
1. Makes sure notebook paths point at the project root (same helper as prior notebooks).
2. Loads backtest results with costs already applied (so PnL is realistic).
3. Creates helper columns: start-of-day equity, end-of-day equity, net PnL, daily return (%), plus convenience flags for "trade" vs "flat" days.
4. Prints a tiny summary so we know the sample size before calculating ratios.


In [ ]:
from pathlib import Path
import sys

ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print('CWD:', Path.cwd())
print('ROOT:', ROOT)
print('Tables dir exists?', (ROOT / 'reports' / 'tables').exists())


In [ ]:
# 5.1 — Load backtest results & compute daily returns

import pandas as pd
import numpy as np

TABLES = ROOT / 'reports' / 'tables'
BT_CSV = TABLES / 'backtest_daily_net.csv'
if not BT_CSV.exists():
    raise FileNotFoundError(f'Missing {BT_CSV}. Re-run Notebook 3 to regenerate the net backtest table.')

daily = pd.read_csv(BT_CSV, parse_dates=['date'])
daily = daily.sort_values('date').reset_index(drop=True)

# Use the shift column (capital before trade) to derive daily returns; avoid division by zero
capital_start = daily['equity_shift'].replace(0, np.nan)
daily['daily_return'] = daily['pnl_usd_net'] / capital_start
daily['had_trade'] = daily['decision'].isin(['long', 'short'])
daily['year'] = daily['date'].dt.year
daily['month'] = daily['date'].dt.month

perf = daily[['date','decision','had_trade','pnl_usd_net','daily_return','equity_shift','equity_net','drawdown_net']].copy()
perf.rename(columns={'equity_shift': 'equity_start', 'equity_net': 'equity_end', 'drawdown_net': 'drawdown'}, inplace=True)
perf.set_index('date', inplace=True)

summary = {
    'start_date': perf.index.min().date(),
    'end_date': perf.index.max().date(),
    'trading_days': int(perf.shape[0]),
    'days_with_trades': int(perf['had_trade'].sum()),
    'days_flat': int((~perf['had_trade']).sum()),
    'ending_equity': float(perf['equity_end'].iloc[-1]),
}
display(pd.Series(summary, name='Backtest sample'))
display(perf.head(3))


### 5.2 — Compounding & annualized return

Annualized returns indicate whether the strategy grows capital fast enough to justify risk. Two complementary looks are summarised below:
- **Daily-compounded annualized return:** product of daily net returns rescaled to 252 trading days.
- **Calendar CAGR:** growth rate implied by beginning and ending equity over the full calendar span.

Both are compared with the >10% per-year benchmark in the metric checklist.


In [ ]:
# 5.2 — Annualized return diagnostics

import matplotlib.pyplot as plt

TRADING_DAYS = 252
daily_returns = perf['daily_return'].dropna()
trading_days = len(daily_returns)
years_span = max((perf.index.max() - perf.index.min()).days / 365.25, trading_days / TRADING_DAYS)

equity_start = perf['equity_start'].iloc[0]
equity_end = perf['equity_end'].iloc[-1]
total_return = equity_end / equity_start - 1
ann_return = (1 + daily_returns).prod() ** (TRADING_DAYS / trading_days) - 1 if trading_days else np.nan
cagr = (equity_end / equity_start) ** (1 / years_span) - 1 if years_span > 0 else np.nan

metrics = pd.DataFrame([
    {"metric": "Total return", "value": total_return, "target": "Positive (>0)", "meets_target": total_return > 0},
    {"metric": "Annualized return (daily compounding)", "value": ann_return, "target": ">=10%", "meets_target": ann_return >= 0.10},
    {"metric": "CAGR (calendar years)", "value": cagr, "target": ">=10%", "meets_target": cagr >= 0.10},
])
display(metrics.style.format({"value": "{:.2%}"}))

fig, ax = plt.subplots(figsize=(9, 4))
ax.plot(perf.index, perf['equity_end'], color='tab:blue', label='Net equity')
ax.set_title('Net equity curve (cost-adjusted)')
ax.set_ylabel('USD')
ax.grid(True, alpha=0.3)
ax.legend(loc='upper left')
plt.tight_layout()
plt.show()



#### 5.2 — Conclusion

Net equity compounds from **$100k → ~$391k** (≈**+291% total return**), so both daily-compounded annualized return (~43%) and calendar CAGR (~41%) easily clear the >10% hurdle. The equity curve still shows multi-month giveback phases, so subsequent sections must confirm that the risk-adjusted profile (Sharpe/Sortino, drawdowns) matches this growth pace.



### 5.3 — Sharpe & Sortino ratios

Risk-adjusted ratios show how efficiently the strategy converts volatility into returns. Two complementary statistics are used:

- **Sharpe ratio** measures excess return per unit of total volatility.  \
  $\displaystyle S = \frac{\sqrt{252} \, \mathbb{E}[R_d - r_f]}{\sigma(R_d - r_f)}$ where $R_d$ is the daily return and $r_f$ is the daily risk-free rate.
- **Sortino ratio** only penalizes downside volatility relative to a target (here 0% daily).  \
  $\displaystyle So = \frac{\sqrt{252} \, \mathbb{E}[R_d - T]}{\sigma(\min(0, R_d - T))}$ with $T$ = target return.

Example: if a strategy averages +0.10% per day with 1.0% standard deviation, then $S ≈ 1.6$; if the downside deviation is 0.4%, then $So ≈ 2.5$. Higher ratios mean the equity curve grows steeply relative to its swings. Targets from the metric checklist: **Sharpe ≥ 1.0** (≥1.5 preferred) and **Sortino ≥ 1.5**.

Here, $r_f$ is set to 3% annualised (approximate short-term US T-bill yield); feel free to adjust if your funding cost differs.


In [ ]:

# 5.3 — Sharpe & Sortino diagnostics

RF_ANNUAL = 0.03  # adjust if you assume a different financing rate
TARGET_DAILY = 0.0

daily_returns = perf['daily_return'].dropna()
rf_daily = RF_ANNUAL / TRADING_DAYS
excess = daily_returns - rf_daily
vol = excess.std(ddof=0)
sharpe = np.sqrt(TRADING_DAYS) * excess.mean() / vol if vol > 0 else np.nan

downside = np.minimum(0, daily_returns - TARGET_DAILY)
downside_std = np.sqrt((downside ** 2).mean())
sortino = (np.sqrt(TRADING_DAYS) * (daily_returns.mean() - TARGET_DAILY) / downside_std
           if downside_std > 0 else np.nan)

win_rate = (daily_returns > TARGET_DAILY).mean()
avg_gain = daily_returns[daily_returns > TARGET_DAILY].mean()
avg_loss = daily_returns[daily_returns <= TARGET_DAILY].mean()

metrics = pd.DataFrame([
    {"metric": "Sharpe ratio", "value": sharpe, "target": ">= 1.0 (>=1.5 ideal)", "meets_target": sharpe >= 1.0},
    {"metric": "Sortino ratio", "value": sortino, "target": ">= 1.5", "meets_target": sortino >= 1.5},
    {"metric": "Daily win rate", "value": win_rate, "target": "Context", "meets_target": np.nan},
    {"metric": "Avg gain (when >0)", "value": avg_gain, "target": "Context", "meets_target": np.nan},
    {"metric": "Avg loss (when <=0)", "value": avg_loss, "target": "Context", "meets_target": np.nan},
])

def _format_value(v):
    if pd.isna(v):
        return '—'
    if abs(v) < 1:
        return f"{v:.2%}"
    return f"{v:.2f}"

metrics['value_display'] = metrics['value'].apply(_format_value)
display(metrics[['metric','value_display','target','meets_target']])

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].hist(daily_returns, bins=40, color='tab:blue', alpha=0.8)
axes[0].axvline(TARGET_DAILY, color='black', linestyle='--', linewidth=1)
axes[0].set_title('Distribution of daily returns (net)')
axes[0].set_xlabel('Daily return')
axes[0].set_ylabel('Frequency')

rolling_sharpe = (daily_returns - rf_daily).rolling(window=63, min_periods=20).apply(
    lambda x: np.sqrt(TRADING_DAYS) * x.mean() / x.std(ddof=0) if x.std(ddof=0) > 0 else np.nan, raw=False
)
axes[1].plot(rolling_sharpe.index, rolling_sharpe, color='tab:orange')
axes[1].axhline(1.0, color='grey', linestyle='--', linewidth=1, label='Sharpe = 1')
axes[1].axhline(1.5, color='grey', linestyle=':', linewidth=1, label='Sharpe = 1.5')
axes[1].set_title('63-day rolling Sharpe (cost-adjusted)')
axes[1].set_ylabel('Ratio')
axes[1].grid(True, alpha=0.3)
axes[1].legend(loc='upper left')

plt.tight_layout()
plt.show()



#### 5.3 — Conclusion

Sharpe ≈ **1.7** and Sortino ≈ **3.5**, so the strategy comfortably exceeds the ≥1 / ≥1.5 thresholds despite posting gains on only ~30% of sessions. Large winners relative to modest average losses drive the high ratios, but the win-rate profile confirms the need for strict discipline during losing streaks. Rolling Sharpe mostly stays above 1 after 2020, suggesting the risk-adjusted edge persists across market regimes sampled here.
